In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from flatsurf import *
import numpy as np
from matplotlib import pyplot as plt
import os
import pwlf
import os
from surface_dynamics.all import *
from Library import *
from Library import Section
import math
from time import time
import copy
from scipy import integrate
import sympy as sym
from sympy import Symbol
from sympy import solve, lambdify
import traceback
import dill
import sys

In [2]:
# list of permutations
def perms_list(n, **kwargs):
    obstructed = []
    p = D.query(nb_squares=n, **kwargs)
    for item in p:
        if not is_unobstructed(item):
            obstructed.append(item)
            for perm in item.teichmueller_curve():
                obstructed.append(perm)
    return obstructed

In [3]:
# take files from saddle.py and load them into notebook
def load_arrays_from_file(file_path):
    # Load arrays from the NumPy file
    arrays_list = np.load(file_path, allow_pickle=True)
    
    # Ensure each element in the list is a NumPy array
    arrays_list = [np.array(array) for array in arrays_list]
    
    return arrays_list

In [4]:
# 7-0, 7-3, 7-7, 7-19, 7-29, 7-41
n_squares = 7
index = 0
j = 0
dx = 0.0005

In [5]:
permutations = perms_list(n_squares)
perm = permutations[index]

vec_file = "vecs" + str(n_squares) + "-" + str(index) + ".npy"
vecs0 = load_arrays_from_file(os.path.join("vecs", vec_file))

with open(os.path.join("results", f"{n_squares} - {index}", "setup.dill"), 'rb') as f:
    loaded_data = dill.load(f)
a,c,e,g = loaded_data

In [6]:
vecs0 = vecs0[:100000]
len(vecs0)

100000

In [7]:
#go thorugh all cusps
dfs = []
dfs1 = []
for j in range(len(a[0])):

    #change back to False in future
    improved = False
    if j == 0:
        improved = True
        
    for i in range(len(a)):

        #get dimensions of section 
        vecs, x_vals, m0, m1, x0, y0, dx_y, z = setup(a[i][j], c[i][j], e[i][j], vecs0, dx, improved)

        if float(z) <= float(1/50000):
            print("too small")
            continue

        print("new")
        t0 = time()
        dfs1.append(winners1(vecs, x_vals, m0, m1, y0, dx, dx_y))
        t1 = time()
        print(t1-t0)
        print()
        print("old")
        t2 = time()
        dfs.append(winners(vecs, x_vals, m0, m1, y0, dx, dx_y))
        t3 = time()
        print(t3-t2)
        print()
        break

new
top done: 0.09845614433288574
side done: 0.14677071571350098
22.062817096710205

old
top done: 26.09654664993286
side done: 24.292970418930054
71.42422080039978

new
top done: 0.10621857643127441
side done: 0.11758875846862793
15.73839783668518

old
top done: 24.503048181533813
side done: 23.98878812789917
63.804540395736694

new
top done: 0.15865278244018555
side done: 0.06875777244567871
15.212928771972656

old
top done: 24.67700719833374
side done: 24.095340490341187
63.94930028915405

new
top done: 0.12073397636413574
side done: 0.06903338432312012
15.175829887390137

old
top done: 24.415343284606934
side done: 24.167150735855103
63.406490087509155

new
top done: 0.06885075569152832
side done: 0.09192013740539551


KeyboardInterrupt: 

In [ ]:
for j in range(len(dfs)):
    df1 = dfs[j]
    df2 = dfs1[j]
    
    for i in range(len(df1)):
        for col in df1.columns:
            if col == "vec":
                if np.array_equal(df1.loc[i, col], df2.loc[i, col]):
                    continue
                else:
                    print(i)
                    break
            else: 
                if df1.loc[i, col] == df2.loc[i, col]:
                    continue
                else: 
                    print(i)
                    break
    print(str(j) + " done")